In [3]:
import http.client
import json
import time
import csv
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import re
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO


class NRGStreamApi:    
    
    def __init__(self,username=None,password=None):
        self.username = 'Fahad'
        self.password = 'ABFAHc2'                
        self.server = 'api.nrgstream.com'        
        self.tokenPath = '/api/security/token'
        self.releasePath = '/api/ReleaseToken'
        self.tokenPayload = f'grant_type=password&username={self.username}&password={self.password}'
        self.tokenExpiry = datetime.now() - timedelta(seconds=60)
        self.accessToken = ""        
        
    def getToken(self):
        try:
            if self.isTokenValid() == False:                             
                headers = { }        
                # Connect to API server to get a token
                conn = http.client.HTTPSConnection(self.server)
                conn.request('POST', self.tokenPath, self.tokenPayload, headers)
                res = conn.getresponse()                
                res_code = res.code
                # Check if the response is good
                
                if res_code == 200:
                    res_data = res.read()
                    # Decode the token into an object
                    jsonData = json.loads(res_data.decode('utf-8'))
                    self.accessToken = jsonData['access_token']                         
                    # Calculate new expiry date
                    self.tokenExpiry = datetime.now() + timedelta(seconds=jsonData['expires_in'])                        
                    #print('token obtained')
                    #print(self.accessToken)
                else:
                    res_data = res.read()
                    print(res_data.decode('utf-8'))
                conn.close()                          
        except Exception as e:
            print("getToken: " + str(e))
            # Release token if an error occured
            self.releaseToken()      

    def releaseToken(self):
        try:            
            headers = {}
            headers['Authorization'] = f'Bearer {self.accessToken}'            
            conn = http.client.HTTPSConnection(self.server)
            conn.request('DELETE', self.releasePath, None, headers)  
            res = conn.getresponse()
            res_code = res.code
            if res_code == 200:   
                # Set expiration date back to guarantee isTokenValid() returns false                
                self.tokenExpiry = datetime.now() - timedelta(seconds=60)
                #print('token released')            
        except Exception as e:
            print("releaseToken: " + str(e))
                    
    def isTokenValid(self):
        if self.tokenExpiry==None:
            return False
        elif datetime.now() >= self.tokenExpiry:            
            return False
        else:
            return True            
    
    def GetStreamDataByStreamId(self,streamIds, fromDate, toDate, dataFormat='csv', dataOption=''):
        stream_data = "" 
        # Set file format to csv or json            
        DataFormats = {}
        DataFormats['csv'] = 'text/csv'
        DataFormats['json'] = 'Application/json'
        
        try:                            
            for streamId in streamIds:            
                # Get an access token            
                self.getToken()    
                if self.isTokenValid():
                    # Setup the path for data request. Pass dates in via function call
                    path = f'/api/StreamData/{streamId}'
                    if fromDate != '' and toDate != '':
                        path += f'?fromDate={fromDate.replace(" ", "%20")}&toDate={toDate.replace(" ", "%20")}'
                    if dataOption != '':
                        if fromDate != '' and toDate != '':
                            path += f'&dataOption={dataOption}'        
                        else:
                            path += f'?dataOption={dataOption}'        
                    
                    # Create request header
                    headers = {}            
                    headers['Accept'] = DataFormats[dataFormat]
                    headers['Authorization']= f'Bearer {self.accessToken}'
                    
                    # Connect to API server
                    conn = http.client.HTTPSConnection(self.server)
                    conn.request('GET', path, None, headers)
                    res = conn.getresponse()        
                    res_code = res.code                    
                    if res_code == 200:   
                        try:
                            print(f'{datetime.now()} Outputing stream {path} res code {res_code}')
                            # output return data to a text file            
                            if dataFormat == 'csv':
                                stream_data += res.read().decode('utf-8').replace('\r\n','\n') 
                            elif dataFormat == 'json':
                                stream_data += json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)
                            conn.close()

                        except Exception as e:
                            print(str(e))            
                            self.releaseToken()
                            return None  
                    else:
                        print(str(res_code) + " - " + str(res.reason) + " - " + str(res.read().decode('utf-8')))
                    
                self.releaseToken()   
                # Wait 1 second before next request
                time.sleep(1)
            return stream_data        
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None
        
        
    def StreamDataOptions(self, streamId, dataFormat='csv'):
        try:      
            DataFormats = {}
            DataFormats['csv'] = 'text/csv'
            DataFormats['json'] = 'Application/json'
            resultSet = {}
            for streamId in streamIds:
                # Get an access token    
                if streamId not in resultSet:
                    self.getToken()                        
                    if self.isTokenValid():                 
                        # Setup the path for data request.
                        path = f'/api/StreamDataOptions/{streamId}'                        
                        # Create request header
                        headers = {}     
                        headers['Accept'] = DataFormats[dataFormat]                                   
                        headers['Authorization'] = f'Bearer {self.accessToken}'
                        # Connect to API server
                        conn = http.client.HTTPSConnection(self.server)
                        conn.request('GET', path, None, headers)
                        res = conn.getresponse()
                        self.releaseToken()       
                        if dataFormat == 'csv':
                            resultSet[streamId] = res.read().decode('utf-8').replace('\r\n','\n') 
                        elif dataFormat == 'json':
                            resultSet[streamId] = json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)                            
                    time.sleep(1)                        
            return resultSet            
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None          
        
        except Exception as e:            
            self.releaseToken()                        
            return str(e)

        
# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')         
# Date range for your data request
# Date format must be mm/dd/yyyy
fromDate = '01/01/2018'
toDate =   '02/11/2021'
# Specify output format - 'csv' or 'json'
dataFormat = 'csv'
# Data Option
dataOption = ''
stream = [3]
for i in stream:
    nrgStreamApi = NRGStreamApi('Username','Password')
    ids= [i]
    stream_data = nrgStreamApi.GetStreamDataByStreamId(ids, fromDate, toDate, dataFormat, dataOption)        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")

    
df = df[14:df.shape[0]]
df.columns = ["Datetime,AIL"]
new = df['Datetime,AIL'].str.split(",", n = 2, expand = True) 
# making separate first name column from new data frame 
df["Datetime"]= new[0] 
# making separate last name column from new data frame 
df["AIL"]= new[1] 
df['AIL'] = pd.to_numeric(df['AIL'],errors='coerce')
df['Datetime']= pd.to_datetime(df['Datetime'])
df= df.drop(columns=['Datetime,AIL'],axis=1)
df = df.reset_index(drop=True)
df

2021-02-12 22:29:49.957202 Outputing stream /api/StreamData/3?fromDate=01/01/2018&toDate=02/11/2021 res code 200


,Datetime,AIL
0,2018-01-01 00:00:00,10221.0
1,2018-01-01 01:00:00,10082.0
2,2018-01-01 02:00:00,9949.0
3,2018-01-01 03:00:00,9886.0
4,2018-01-01 04:00:00,9930.0
...,...,...
27283,2021-02-10 19:00:00,11485.0
27284,2021-02-10 20:00:00,11364.0
27285,2021-02-10 21:00:00,11195.0
27286,2021-02-10 22:00:00,10961.0


In [12]:
df['hour_of_day']= df['Datetime'].dt.hour

#------------------------------
import numpy as np
# create a list of our conditions
conditions = [
    (df['hour_of_day'] < 7),
    (df['hour_of_day'] >= 7) & (df['hour_of_day'] <= 19),
    (df['hour_of_day'] > 19)
    ]

# create a list of the values we want to assign for each condition
values = [1, 0, 1]

# create a new column and use np.select to assign values to it using our lists as arguments
df['off_peak'] = np.select(conditions, values)

conditions = [
    (df['hour_of_day'] < 7),
    (df['hour_of_day'] >= 7) & (df['hour_of_day'] <= 19),
    (df['hour_of_day'] > 19)
    ]

# create a list of the values we want to assign for each condition
values = [0, 1, 0]

# create a new column and use np.select to assign values to it using our lists as arguments
df['on_peak'] = np.select(conditions, values)

#----------------------
df['just_date'] = df['Datetime'].dt.date
dates = df['just_date']
day = pd.Series([d.timetuple().tm_yday for d in dates])
df['day'] = day

#------------------------

df['sin.day'] = np.sin(day*2*np.pi/365 + np.pi/4)
df['cos.day'] = np.cos(day*2*np.pi/365 + np.pi/4)
df['sin.hour'] = np.sin(df['hour_of_day']*2*np.pi/24)
df['cos.hour'] = np.cos(df['hour_of_day']*2*np.pi/24)
#-------------------------

weekdays = [d.weekday() for d in dates]
df['weekend'] = [1 if d >= 5 else 0 for d in weekdays]
for i, s in enumerate(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',
            'sunday']):
    df[s] = [1 if d == i else 0 for d in weekdays]

#-----------------------------------------

from sklearn.preprocessing import OneHotEncoder
month = [[d.timetuple().tm_mon-1] for d in dates]
month_bin = OneHotEncoder(dtype=int, sparse=False).fit_transform(month)
for i in range(12):
    df['month_%d' % i] = month_bin[:, i]
    
for i in range(24):
    df['hour_%d' % i] = np.where(df['hour_of_day']==i, 1, 0)

In [13]:
xls = pd.ExcelFile('edmonton_sunrise_sunset.xls')
dfess_2015 = pd.read_excel(xls, '2015') #df= dafaframe, e= edmonston, ss= sunrise and sunset 
dfess_2016 = pd.read_excel(xls, '2016')

df['year']= df['Datetime'].dt.year
dfess_2015 = dfess_2015[['Sunrise_hr', 'Sunset_hr']]
dfess_2016 = dfess_2016[['Sunrise_hr', 'Sunset_hr']]
df['sunlight_avaialbility'] =  0

for i in range(0,df.shape[0]):
    if (df.iloc[i,df.columns.get_loc('year')]%4 ==0 and df.iloc[i,df.columns.get_loc('year')]%100 !=0):
        criteria = df.iloc[i,df.columns.get_loc('day')] #day_of_year
        sunrise  = dfess_2016.iloc[criteria-1,0] #sunrise
        sunset   = dfess_2016.iloc[criteria-1,1] #sunset
        if (df.iloc[i,df.columns.get_loc('hour_of_day')]>= sunrise) and (df.iloc[i,df.columns.get_loc('hour_of_day')] <= sunset):
            df.iloc[i,df.columns.get_loc('sunlight_avaialbility')] = 1
    else:
        criteria = df.iloc[i,df.columns.get_loc('day')] #day_of_year
        sunrise  = dfess_2015.iloc[criteria-1,0]#sunrise
        sunset   = dfess_2015.iloc[criteria-1,1] #sunset
        if (df.iloc[i,df.columns.get_loc('hour_of_day')]>= sunrise) and (df.iloc[i,df.columns.get_loc('hour_of_day')] <= sunset):
            df.iloc[i,df.columns.get_loc('sunlight_avaialbility')] = 1
            
df

,Datetime,AIL,hour_of_day,off_peak,on_peak,just_date,day,sin.day,cos.day,sin.hour,...,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,year,sunlight_avaialbility
0,2018-01-01 00:00:00,10221.0,0,1,0,2018-01-01,1,0.719174,0.694830,0.000000,...,0,0,0,0,0,0,0,0,2018,0
1,2018-01-01 01:00:00,10082.0,1,1,0,2018-01-01,1,0.719174,0.694830,0.258819,...,0,0,0,0,0,0,0,0,2018,0
2,2018-01-01 02:00:00,9949.0,2,1,0,2018-01-01,1,0.719174,0.694830,0.500000,...,0,0,0,0,0,0,0,0,2018,0
3,2018-01-01 03:00:00,9886.0,3,1,0,2018-01-01,1,0.719174,0.694830,0.707107,...,0,0,0,0,0,0,0,0,2018,0
4,2018-01-01 04:00:00,9930.0,4,1,0,2018-01-01,1,0.719174,0.694830,0.866025,...,0,0,0,0,0,0,0,0,2018,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27283,2021-02-10 19:00:00,11485.0,19,0,1,2021-02-10,41,0.996832,0.079532,-0.965926,...,0,0,0,1,0,0,0,0,2021,0
27284,2021-02-10 20:00:00,11364.0,20,1,0,2021-02-10,41,0.996832,0.079532,-0.866025,...,0,0,0,0,1,0,0,0,2021,0
27285,2021-02-10 21:00:00,11195.0,21,1,0,2021-02-10,41,0.996832,0.079532,-0.707107,...,0,0,0,0,0,1,0,0,2021,0
27286,2021-02-10 22:00:00,10961.0,22,1,0,2021-02-10,41,0.996832,0.079532,-0.500000,...,0,0,0,0,0,0,1,0,2021,0


In [16]:
df['AIL_previous_hour']  = df['AIL'].shift(1)     # col_index = 57
df['AIL_24h_lagged']     = df['AIL'].shift(24)    # col_index = 58
df['AIL_2day_lagged']    = df['AIL'].shift(24*2)  # col_index = 59
df['AIL_3day_lagged']    = df['AIL'].shift(24*3)  # col_index = 60
df['AIL_4day_lagged']    = df['AIL'].shift(24*4)  # col_index = 61
df['AIL_5day_lagged']    = df['AIL'].shift(24*5)  # col_index = 62
df['AIL_6day_lagged']    = df['AIL'].shift(24*6)  # col_index = 63
df['AIL_oneweek_lagged'] = df['AIL'].shift(24*7)  #col_index = 64

In [17]:
# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')         
# Date range for your data request
# Date format must be mm/dd/yyyy
fromDate = '12/31/2017'
toDate =   '12/31/2017'
# Specify output format - 'csv' or 'json'
dataFormat = 'csv'
# Data Option
dataOption = ''
stream = [3]
for i in stream:
    nrgStreamApi = NRGStreamApi('Username','Password')
    ids= [i]
    stream_data = nrgStreamApi.GetStreamDataByStreamId(ids, fromDate, toDate, dataFormat, dataOption)        
    STREAM_DATA = StringIO(stream_data)
    test_df = pd.read_csv(STREAM_DATA, sep=";")

    
test_df = test_df[14:test_df.shape[0]]
test_df.columns = ["Datetime,AIL"]
new = test_df['Datetime,AIL'].str.split(",", n = 2, expand = True) 
# making separate first name column from new data frame 
test_df["Datetime"]= new[0] 
# making separate last name column from new data frame 
test_df["AIL"]= new[1] 
test_df['AIL'] = pd.to_numeric(df['AIL'],errors='coerce')
test_df['Datetime']= pd.to_datetime(df['Datetime'])
test_df= test_df.drop(columns=['Datetime,AIL'],axis=1)
test_df = test_df.reset_index(drop=True)

df.iloc[0,df.columns.get_loc('AIL_previous_hour')] = test_df.iloc[23,1]

df.iloc[:,df.columns.get_loc('AIL_previous_hour')]

2021-02-12 22:49:03.240190 Outputing stream /api/StreamData/3?fromDate=12/31/2017&toDate=12/31/2017 res code 200


0        10571.0
1        10221.0
2        10082.0
3         9949.0
4         9886.0
          ...   
27283    11665.0
27284    11485.0
27285    11364.0
27286    11195.0
27287    10961.0
Name: AIL_previous_hour, Length: 27288, dtype: float64

In [37]:
# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')         
# Date range for your data request
# Date format must be mm/dd/yyyy
fromDate = '01/01/2018'
toDate =   '01/08/2018'
# Specify output format - 'csv' or 'json'
dataFormat = 'csv'
# Data Option
dataOption = ''
stream = [3]
for i in stream:
    nrgStreamApi = NRGStreamApi('Username','Password')
    ids= [i]
    stream_data = nrgStreamApi.GetStreamDataByStreamId(ids, fromDate, toDate, dataFormat, dataOption)        
    STREAM_DATA = StringIO(stream_data)
    test_df = pd.read_csv(STREAM_DATA, sep=";")
    #test_df = test_df.iloc[1:14+7*24+1]
    #test_df = test_df[0: 183]
    test_df.columns = ["Datetime,AIL"]
    new = test_df['Datetime,AIL'].str.split(",", n = 2, expand = True) 
    # making separate first name column from new data frame 
    test_df["Datetime"]= new[0] 
    # making separate last name column from new data frame 
    test_df["AIL"]= new[1] 
    test_df['AIL'] = pd.to_numeric(df['AIL'],errors='coerce')
    test_df['Datetime']= pd.to_datetime(df['Datetime'])
    test_df= test_df.drop(columns=['Datetime,AIL'],axis=1)
    test_df = test_df.reset_index(drop=True)

#test_df
df.iloc[0:7*24, df.columns.get_loc('AIL_oneweek_lagged')] = test_df.iloc[0:24*7,1]
df.iloc[0:6*24, df.columns.get_loc('AIL_6day_lagged')]    = test_df.iloc[0:24*6,1]
df.iloc[0:5*24, df.columns.get_loc('AIL_5day_lagged')]    = test_df.iloc[0:24*5,1]
df.iloc[0:4*24, df.columns.get_loc('AIL_4day_lagged')]    = test_df.iloc[0:24*4,1]
df.iloc[0:3*24, df.columns.get_loc('AIL_3day_lagged')]    = test_df.iloc[0:24*3,1]
df.iloc[0:2*24, df.columns.get_loc('AIL_2day_lagged')]    = test_df.iloc[0:24*2,1]
df.iloc[0:1*24, df.columns.get_loc('AIL_24h_lagged')]     = test_df.iloc[0:24*1,1]

2021-02-12 23:17:06.799793 Outputing stream /api/StreamData/3?fromDate=01/01/2018&toDate=01/08/2018 res code 200


In [104]:
'''
import holidays
hl_list       = holidays.CA(years=[2018,2019,2020,2021], prov = 'AB').items()
df_hl         = pd.DataFrame(hl_list)
df_hl.columns = ['date','title']
df_hl['date'] = pd.to_datetime(df_hl['date'])
#df_hl         = df_hl.sort_values('date')
df_hl['date'] = df_hl['date'].dt.date
df_hl

#df['holiday'] = 0
#pd.to_datetime(df.iloc[i,df.columns.get_loc('just_date')]) == df_hl[j, df_hl.columns.get_loc('date')]

#for i in range(0, df.shape[0]):
#    for j in range(0,df_hl.shape[0]):
#        if pd.to_datetime(df.iloc[i,df.columns.get_loc('just_date')]) == df_hl[j, df_hl.columns.get_loc('date')]:
#            df.iloc[i,df.columns.get_loc('holiday')]  = 1
#df

'''

"\nimport holidays\nhl_list       = holidays.CA(years=[2018,2019,2020,2021], prov = 'AB').items()\ndf_hl         = pd.DataFrame(hl_list)\ndf_hl.columns = ['date','title']\ndf_hl['date'] = pd.to_datetime(df_hl['date'])\n#df_hl         = df_hl.sort_values('date')\ndf_hl['date'] = df_hl['date'].dt.date\ndf_hl\n\n#df['holiday'] = 0\n#pd.to_datetime(df.iloc[i,df.columns.get_loc('just_date')]) == df_hl[j, df_hl.columns.get_loc('date')]\n\n#for i in range(0, df.shape[0]):\n#    for j in range(0,df_hl.shape[0]):\n#        if pd.to_datetime(df.iloc[i,df.columns.get_loc('just_date')]) == df_hl[j, df_hl.columns.get_loc('date')]:\n#            df.iloc[i,df.columns.get_loc('holiday')]  = 1\n#df\n\n"

In [105]:
'''
df['monday_holiday']    = df['monday'] * df['holiday']
df['tuesday_holiday']   = df['tuesday']* df['holiday'] 
df['wednesday_holiday'] = df['wednesday'] * df['holiday']
df['thursday_holiday']  = df['thursday'] * df['holiday']
df['friday_holiday']    = df['friday'] * df['holiday']
df['weekend_holiday'] = df['weekend'] * df['holiday']


df['month0_mondayholiday'] = df['month_0']*df['monday_holiday']
df['month1_mondayholiday'] = df['month_1']*df['monday_holiday']
df['month2_mondayholiday'] = df['month_2']*df['monday_holiday']
df['month3_mondayholiday'] = df['month_3']*df['monday_holiday']
df['month4_mondayholiday'] = df['month_4']*df['monday_holiday']
df['month5_mondayholiday'] = df['month_5']*df['monday_holiday']
df['month6_mondayholiday'] = df['month_6']*df['monday_holiday']
df['month7_mondayholiday'] = df['month_7']*df['monday_holiday']
df['month8_mondayholiday'] = df['month_8']*df['monday_holiday']
df['month9_mondayholiday'] = df['month_9']*df['monday_holiday']
df['month10_mondayholiday'] = df['month_10']*df['monday_holiday']
df['month11_mondayholiday'] = df['month_11']*df['monday_holiday']

df['month0_fridayholiday'] = df['month_0']*df['friday_holiday']
df['month1_fridayholiday'] = df['month_1']*df['friday_holiday']
df['month2_fridayholiday'] = df['month_2']*df['friday_holiday']
df['month3_fridayholiday'] = df['month_3']*df['friday_holiday']
df['month4_fridayholiday'] = df['month_4']*df['friday_holiday']
df['month5_fridayholiday'] = df['month_5']*df['friday_holiday']
df['month6_fridayholiday'] = df['month_6']*df['friday_holiday']
df['month7_fridayholiday'] = df['month_7']*df['friday_holiday']
df['month8_fridayholiday'] = df['month_8']*df['friday_holiday']
df['month9_fridayholiday'] = df['month_9']*df['friday_holiday']
df['month10_fridayholiday'] = df['month_10']*df['friday_holiday']
df['month11_fridayholiday'] = df['month_11']*df['friday_holiday']

df['sunlight_mondayholiday']  = df['sunlight_avaialbility']*df['monday_holiday']
df['sunlight_tuesdayholiday'] = df['sunlight_avaialbility']*df['tuesday_holiday']
df['sunlight_tuesdayholiday'] = df['sunlight_avaialbility']*df['wednesday_holiday']
df['sunlight_tuesdayholiday'] = df['sunlight_avaialbility']*df['thursday_holiday']
df['sunlight_tuesdayholiday'] = df['sunlight_avaialbility']*df['friday_holiday']

df

'''



"\ndf['monday_holiday']    = df['monday'] * df['holiday']\ndf['tuesday_holiday']   = df['tuesday']* df['holiday'] \ndf['wednesday_holiday'] = df['wednesday'] * df['holiday']\ndf['thursday_holiday']  = df['thursday'] * df['holiday']\ndf['friday_holiday']    = df['friday'] * df['holiday']\ndf['weekend_holiday'] = df['weekend'] * df['holiday']\n\n\ndf['month0_mondayholiday'] = df['month_0']*df['monday_holiday']\ndf['month1_mondayholiday'] = df['month_1']*df['monday_holiday']\ndf['month2_mondayholiday'] = df['month_2']*df['monday_holiday']\ndf['month3_mondayholiday'] = df['month_3']*df['monday_holiday']\ndf['month4_mondayholiday'] = df['month_4']*df['monday_holiday']\ndf['month5_mondayholiday'] = df['month_5']*df['monday_holiday']\ndf['month6_mondayholiday'] = df['month_6']*df['monday_holiday']\ndf['month7_mondayholiday'] = df['month_7']*df['monday_holiday']\ndf['month8_mondayholiday'] = df['month_8']*df['monday_holiday']\ndf['month9_mondayholiday'] = df['month_9']*df['monday_holiday']\ndf

In [106]:
df.to_csv("training_data_2018_2020_feb_13_20201.csv")